In [1]:
# library dependeceis
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

# online
import duckdb
import requests
import urllib.parse
import re


In [2]:
import os
import scipy.stats
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.neighbors
import duckdb

from Bio import pairwise2
from Bio.Seq import Seq

In [3]:
# seaborn plot configurations
sns.set_context('talk')
sns.set_style('whitegrid')
palette = sns.color_palette('colorblind')

In [4]:
# CSV reading
#, index_col=0
df = pd.read_csv("/Users/amin/ValidProt/data/Sample.csv")
df

,Unnamed: 0,prot_pair_index,meso_seq,thermo_seq,meso_ogt,thermo_ogt,scaled_local_symmetric_percent_id,local_E_value,scaled_local_query_percent_id,local_gap_compressed_percent_id
0,1256842,126227630,MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNH...,MLLSDRDLRKELESGRLELDPFDPAMLQPSSIDVRLDRFFRVFDNT...,27.5,45.0,0.777202,0.0,0.773196,0.802139
1,1456567,169784592,MRFEGTSGYVATDDLKVAVNAAIALERPLLVKGEPGTGKTVLAVEV...,MKFTGSDSYVATEDLMIAVNAAVTLERPLLVKGEPGTGKTELARQV...,30.0,54.0,0.782143,0.0,0.784946,0.784946
2,874464,31933768,MAYETINVDVQDHVCLIKLHRPEALNALNAALVSELCTALEEADAS...,MAYKTIIVEIEDHVALIKLNRPEALNALNSELLGELAQAVTEADAN...,19.5,54.0,0.775194,0.0,0.775194,0.775194
3,560201,32409414,MAIRKYKPTTPGRRGSSVADFAEITRSTPEKSLLRPLSKTGGRNNQ...,MGIRKYKPTTPGRRGASVADFVELTRREPEKSLLRPLPKKGGRNNR...,28.0,52.5,0.787770,0.0,0.790614,0.802198
4,33257,175862226,MLQRLQDRVAVVTGGGSGIGLATVRRFAAEGAKVVVADIDAAAGEA...,MSEDIICRRLTGRTAVVTGAGSGIGLASARRLASEGANVVCADVDE...,28.0,45.0,0.788350,0.0,0.780769,0.802372
...,...,...,...,...,...,...,...,...,...,...
9995,458544,42061208,MRFVIARCQVDYVGRLTAHLPMANRLVMVKSDGSVLVHSDGGSYKP...,MRLVIARCQVDYVGRLTAHLPMAQRLLLIKADGSVSVHSDDRAYKP...,28.0,45.0,0.872146,0.0,0.872146,0.872146
9996,469612,165123678,MKPIVGSIVALITPMHEDGSVDYPALRKLIDWHIAEGTDCIGVVGT...,MTSSRVTLTGSIVALVTPMHEDGSVDYPTLRKLIDWHIAQGTDCIS...,30.0,48.0,0.764805,0.0,0.758389,0.776632
9997,226964,71308777,MSFFAPKTVVSAHCDLPCGVYDPAQARIEAESIKAVAEKYQANTDP...,MLSRLFAPTVEVSAHCDLPCGVYDPAQARIEAQSIKAIIEKYHASD...,30.0,52.5,0.777358,0.0,0.768657,0.804688
9998,123183,137720058,MRLVIARCSVDYVGRLTAHLPMATRLLLVKADGSVSVHADDRAYKP...,MRLVIARCQVDYHGRLTAHLPMATRLVLIKADGSVSIHSDDRAYKP...,32.0,50.0,0.737557,0.0,0.744292,0.740909


# Pairing

In [5]:
df["meso_seq"].iloc[0]+df["thermo_seq"].iloc[0]

'MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTSMLLSDRDLRKELESGRLELDPFDPAMLQPSSIDVRLDRFFRVFDNTKYTHIDPSIQQDELTSLVEVGDGDEPFVLHPGEFVLGSTYEMVTLPDDLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHITLELSNVANLPITLWPGMKIGQLCLFRLTSAAEHPYGSARAGSRYQHQRGPTPSRAYLNFSRIDTRR'

# We need that in the future

In [6]:
from Bio.Seq import Seq

seq_list = []
for i in range(min(len(df['meso_seq']), len(df['thermo_seq']))):
    seq1 = Seq(df['meso_seq'][i])
    seq2 = Seq(df['thermo_seq'][i])
    seq12 = seq1 + seq2
    seq_list.append((seq1, seq2, seq12))
samples = pd.DataFrame(seq_list, columns=['meso_seq', 'thermo_seq', 'seq12'])
print(samples)

                                               meso_seq  \
0     (M, L, L, S, D, R, D, L, V, S, E, I, K, S, G, ...   
1     (M, R, F, E, G, T, S, G, Y, V, A, T, D, D, L, ...   
2     (M, A, Y, E, T, I, N, V, D, V, Q, D, H, V, C, ...   
3     (M, A, I, R, K, Y, K, P, T, T, P, G, R, R, G, ...   
4     (M, L, Q, R, L, Q, D, R, V, A, V, V, T, G, G, ...   
...                                                 ...   
9995  (M, R, F, V, I, A, R, C, Q, V, D, Y, V, G, R, ...   
9996  (M, K, P, I, V, G, S, I, V, A, L, I, T, P, M, ...   
9997  (M, S, F, F, A, P, K, T, V, V, S, A, H, C, D, ...   
9998  (M, R, L, V, I, A, R, C, S, V, D, Y, V, G, R, ...   
9999  (M, T, S, P, K, Q, Q, Q, C, G, F, V, A, V, I, ...   

                                             thermo_seq  \
0     (M, L, L, S, D, R, D, L, R, K, E, L, E, S, G, ...   
1     (M, K, F, T, G, S, D, S, Y, V, A, T, E, D, L, ...   
2     (M, A, Y, K, T, I, I, V, E, I, E, D, H, V, A, ...   
3     (M, G, I, R, K, Y, K, P, T, T, P, G, R, R, G, ...

# Alignment (we need that in the future)

In [7]:
# Creating sample sequences
seqpair = []
for i in range(0,10000):
    seq1 = Seq(df['meso_seq'][i])
    seq2 = Seq(df['thermo_seq'][i])
    seqpair.append((seq1, seq2))

# Finding similarities
results = []
for seq1, seq2 in seqpair:
    alignments = pairwise2.align.globalxx(seq1, seq2)

    # Adding results to a list
    for match in alignments:
        results.append({
            'seqA': match.seqA,
            'seqB': match.seqB,
            'score': match.score,
            'start': match.start,
            'end': match.end
        })

# Creating a pandas DataFrame
alignment = pd.DataFrame(results)

# Printing the DataFrame
alignment


,seqA,seqB,score,start,end
0,MLLSDRDLVSEI-KSGD-L-S--LE--PFE-PAL-LQPSSIDVRLD...,MLLSDRDL----RK---ELESGRLELDPF-DPA-MLQPSSIDVRLD...,154.0,0,232
1,MLLSDRDLVSEIKSGD-L-S--LE--PFE-PAL-LQPSSIDVRLDR...,MLLSDRDL---RK---ELESGRLELDPF-DPA-MLQPSSIDVRLDR...,154.0,0,231
2,MLLSDRDLVSEIKSGD-L-S--LE--PFE-PAL-LQPSSIDVRLDR...,MLLSDRDL--R-K---ELESGRLELDPF-DPA-MLQPSSIDVRLDR...,154.0,0,231
3,MLLSDRDLVSEIKSGD-L-S--LE--PFE-PAL-LQPSSIDVRLDR...,MLLSDRDL-R--K---ELESGRLELDPF-DPA-MLQPSSIDVRLDR...,154.0,0,231
4,MLLSDRDLVSEIKSGD-L-S--LE--PFE-PAL-LQPSSIDVRLDR...,MLLSDRDLR---K---ELESGRLELDPF-DPA-MLQPSSIDVRLDR...,154.0,0,231
...,...,...,...,...,...
3529514,MTSPKQQQCGFVAVI-GAPNAGKSTLVN-AMVGQKVAIVSAKAQTT...,MT-EK---CGLVAV-LGAPNAGKSTLVNTL-VGQKVAIVSAKAQTT...,227.0,0,367
3529515,MTSPKQQQCGFVAVI-GAPNAGKSTLVN-AMVGQKVAIVSAKAQTT...,MTE-K---CGLVAV-LGAPNAGKSTLVNTL-VGQKVAIVSAKAQTT...,227.0,0,367
3529516,MTSP-KQQQCGF-VAVIGAPNAGKSTLVN-AMVGQKVAIVSAKAQT...,MT--EK---CG-LVAVLGAPNAGKSTLVNTL-VGQKVAIVSAKAQT...,227.0,0,368
3529517,MTSPKQQQCGF-VAVIGAPNAGKSTLVN-AMVGQKVAIVSAKAQTT...,MT-EK---CG-LVAVLGAPNAGKSTLVNTL-VGQKVAIVSAKAQTT...,227.0,0,367


# API: with protein ID

In [8]:
import requests

accession = "IPR002394"
url = f"https://www.ebi.ac.uk/interpro/api/entry/interpro/{accession}"
response = requests.get(url)

if response.ok:
    data2 = response.json()
    # process the data as needed
else:
    print("Request failed")

    
print(data2)

{'metadata': {'accession': 'IPR002394', 'entry_id': None, 'type': 'family', 'go_terms': [{'identifier': 'GO:0005216', 'name': 'ion channel activity', 'category': {'code': 'F', 'name': 'molecular_function'}}, {'identifier': 'GO:0022848', 'name': 'acetylcholine-gated cation-selective channel activity', 'category': {'code': 'F', 'name': 'molecular_function'}}, {'identifier': 'GO:0006811', 'name': 'ion transport', 'category': {'code': 'P', 'name': 'biological_process'}}, {'identifier': 'GO:0016020', 'name': 'membrane', 'category': {'code': 'C', 'name': 'cellular_component'}}, {'identifier': 'GO:0045211', 'name': 'postsynaptic membrane', 'category': {'code': 'C', 'name': 'cellular_component'}}], 'source_database': 'interpro', 'member_databases': {'prints': {'PR00254': 'NICOTINICR'}}, 'integrated': None, 'hierarchy': {'accession': 'IPR006201', 'name': 'Neurotransmitter-gated ion-channel', 'type': 'Family', 'children': [{'accession': 'IPR002394', 'name': 'Nicotinic acetylcholine receptor', 't

# HMMER API: by sequence 

### For one sequence `sequence = df['meso_seq'][0]`


### Text results:

In [9]:
# for one sequence - print the text results

sequence = df['meso_seq'][0]
url = 'https://www.ebi.ac.uk/Tools/hmmer/search/hmmscan'
headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
data = {'hmmdb': 'pfam',
        'seq': f'>seq\n{sequence}'
               }

data = urllib.parse.urlencode(data).encode('ascii')

response = requests.post(url, headers=headers, data=data, allow_redirects=False)

redirect_url = response.headers.get('Location')


if redirect_url is None:
    print("Error: No redirect URL found in response.")
else:
    response2 = requests.get(redirect_url, headers=headers)
    

response2.text

'{"results":{"hits":[{"flags":3,"nregions":1,"ndom":2,"name":"DCD","score":46.2,"bias":"0.0","taxid":"PF06559.14","acc":"PF06559.14","domains":[{"alisqacc":"","aliIdCount":35,"alirfline":"","is_included":1,"alihmmname":"DCD","bitscore":28.6933517456055,"display":0,"ievalue":"6.0e-07","alisqto":111,"aliSim":0.660550458715596,"jali":111,"bias":"0.01","ienv":2,"cevalue":6.1e-11,"significant":1,"alimline":"l+++++ + i++g + + ep+e   +qP+s+D+rL  + +Rv    l  + +  ae+ ++l+ + e++  e  vl  g   l+  le ++L ++l+++ + kss GRl ++t","alihmmfrom":5,"clan":"CL0153","aliL":192,"alihindex":"3134","is_reported":1,"alintseq":"","jenv":114,"alimmline":"","alihmmacc":"PF06559.14","oasc":"0.83","aliaseq":"LSDRDLVSEIKSGDL-SLEPFEPALLQPSSIDVRLD-RFFRVFNNHLYTHID-PAEQQDDLTAEvEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLT","alihmmto":115,"aliId":0.321100917431193,"alippline":"677777777888888.5679***************7.57********97655.55666777653145555778***********************************9998","alimodel":"lpaqaiealiaegaiaaee

### Results in table

In [10]:
# for one sequence - print the results in table

sequence = df['meso_seq'][4]
url = 'https://www.ebi.ac.uk/Tools/hmmer/search/hmmscan'
headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
data = {'hmmdb': 'pfam',
        'seq': f'>seq\n{sequence}'
               }

data = urllib.parse.urlencode(data).encode('ascii')

response = requests.post(url, headers=headers, data=data, allow_redirects=False)

redirect_url = response.headers.get('Location')


if redirect_url is None:
    print("Error: No redirect URL found in response.")
else:
    response2 = requests.get(redirect_url, headers=headers)
    #print(response2.text)
    results = response2.json()
    hits = results['results']['hits']
    dff = pd.json_normalize(hits, 'domains', ['acc', 'name', 'score', 'evalue', 'pvalue', 'desc'])

dff
    

,alisqacc,aliIdCount,alirfline,is_included,alihmmname,bitscore,display,ievalue,alisqto,aliSim,...,act_site.adh_short_C2.154.Y,act_site.adh_short.158.K,act_site.adh_short.154.Y,act_site.Epimerase.117.Y,acc,name,score,evalue,pvalue,desc
0,,92,,1,adh_short_C2,233.947098,1.0,0.0,248,0.861472,...,Similarity to Q12634,NaN,NaN,NaN,PF13561.9,adh_short_C2,234.2,0.0,-168.330861,Enoyl-(Acyl carrier protein) reductase
1,,77,,1,adh_short,173.691223,0.0,0.0,198,0.847368,...,NaN,Similarity to Q06136,Similarity to P0AET8,NaN,PF00106.28,adh_short,173.9,0.0,-126.327021,short chain dehydrogenase
2,,19,,0,Epimerase,20.300087,NaN,0.00034,74,0.765625,...,NaN,NaN,NaN,NaN,PF01370.24,Epimerase,23.3,0.000041,-19.98008,NAD dependent epimerase/dehydratase family
3,,5,,0,Epimerase,1.288505,NaN,220,130,0.809524,...,NaN,NaN,NaN,Similarity to Q7CRQ0,PF01370.24,Epimerase,23.3,0.000041,-19.98008,NAD dependent epimerase/dehydratase family


### All data from one row of the results table

In [11]:
dff.iloc[0]

alisqacc                                                                        
aliIdCount                                                                    92
alirfline                                                                       
is_included                                                                    1
alihmmname                                                          adh_short_C2
bitscore                                                              233.947098
display                                                                      1.0
ievalue                                                                      0.0
alisqto                                                                      248
aliSim                                                                  0.861472
jali                                                                         248
bias                                                                        1.77
ienv                        

### By dropping columns, we can have only the needed columns. 

In [12]:
dff.shape

(4, 52)

In [13]:
#df = df.drop(['age', 'height'], axis=1)
dff_new = dff.drop(['alimmline','alirfline','alisqacc'], axis=1)
dff_new.shape

(4, 49)

### For a loop for an extensive dataset 

In [14]:

#for i in range(len(df['meso_seq'])):

for i in range(0,5):
    sequence = df['meso_seq'][i]
    url = 'https://www.ebi.ac.uk/Tools/hmmer/search/hmmscan'
    headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
    data = {'hmmdb': 'pfam',
            'seq': f'>seq\n{sequence}'
                   }

    data = urllib.parse.urlencode(data).encode('ascii')

    response = requests.post(url, headers=headers, data=data, allow_redirects=False)

    redirect_url = response.headers.get('Location')


    if redirect_url is None:
        print("Error: No redirect URL found in response.")
    else:
        response2 = requests.get(redirect_url, headers=headers)
        print(response2.text)

{"results":{"hits":[{"flags":3,"nregions":1,"ndom":2,"name":"DCD","score":46.2,"bias":"0.0","taxid":"PF06559.14","acc":"PF06559.14","domains":[{"alisqacc":"","aliIdCount":35,"alirfline":"","is_included":1,"alihmmname":"DCD","bitscore":28.6933517456055,"display":0,"ievalue":"6.0e-07","alisqto":111,"aliSim":0.660550458715596,"jali":111,"bias":"0.01","ienv":2,"cevalue":6.1e-11,"significant":1,"alimline":"l+++++ + i++g + + ep+e   +qP+s+D+rL  + +Rv    l  + +  ae+ ++l+ + e++  e  vl  g   l+  le ++L ++l+++ + kss GRl ++t","alihmmfrom":5,"clan":"CL0153","aliL":192,"alihindex":"3134","is_reported":1,"alintseq":"","jenv":114,"alimmline":"","alihmmacc":"PF06559.14","oasc":"0.83","aliaseq":"LSDRDLVSEIKSGDL-SLEPFEPALLQPSSIDVRLD-RFFRVFNNHLYTHID-PAEQQDDLTAEvEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLT","alihmmto":115,"aliId":0.321100917431193,"alippline":"677777777888888.5679***************7.57********97655.55666777653145555778***********************************9998","alimodel":"lpaqaiealiaegaiaaeep

{"results":{"hits":[{"flags":3,"nregions":1,"ndom":1,"name":"ECH_1","score":"276.0","bias":"2.8","taxid":"PF00378.23","acc":"PF00378.23","domains":[{"alisqacc":"","aliIdCount":117,"alirfline":"","is_included":1,"alihmmname":"ECH_1","bitscore":275.810150146484,"display":1,"ievalue":3.1e-82,"alisqto":258,"aliSim":0.870967741935484,"jali":258,"bias":"2.79","ienv":9,"cevalue":3.1e-86,"act_site":{"ECH_1":{"112":{"E":"Similarity to Q5LLW6"},"132":{"E":"Similarity to Q5LLW6"}}},"significant":1,"alimline":"++ v++ikl+rPea+Nal+a+l+ El  ale+++ + +vr +vLtG++kaFaaGad+kem e   ++ +++++++++   +++  + kp+iaav G+a+gGG+ela+ cD++iaa++a+fg++e++lG+++g ggtq+l+r vGksk + + l+g+ ++a+ea+++Glv++vvp+++l++ea+++a+k+aeks++ ++++kea+n+ +e +l e++ +e++ f+s+f+++d keG+ aflEkr+++f+dk","alihmmfrom":3,"clan":"CL0127","aliL":258,"alihindex":"8476","is_reported":1,"alintseq":"","jenv":258,"alimmline":"","alihmmacc":"PF00378.23","oasc":"0.97","aliaseq":"QDHVCLIKLHRPEALNALNAALVSELCTALEEADASDKVRCIVLTGSEKAFAAGADIKEMSEM--SFTDVYSSNLFA

{"results":{"hits":[{"flags":3,"nregions":1,"ndom":1,"name":"adh_short_C2","score":234.2,"bias":"1.8","taxid":"PF13561.9","acc":"PF13561.9","domains":[{"alisqacc":"","aliIdCount":92,"alirfline":"","is_included":1,"alihmmname":"adh_short_C2","bitscore":233.94709777832,"display":1,"ievalue":1.9e-69,"alisqto":248,"aliSim":0.861471861471861,"jali":248,"bias":"1.77","ienv":12,"cevalue":2.9e-73,"act_site":{"adh_short_C2":{"154":{"Y":"Similarity to Q12634"}}},"significant":1,"alimline":"sgIG a+++++a+eGa+vv++d++ +a    e++a+e+g  ++++DVtde+qvea+++++v+++g+lDi++nnagi+p+ + ++l+t  e ++r+++vnl++++l++k+a++ ++++g gs++n++s+ a  + +  +  y+a+K+++ a++r l v+++++giRvNa +pgp++Tp+ +++ ++++e  ++   ++p+gr+++pee+a+a++fLasd++s+it +++ vDgG++","alihmmfrom":4,"clan":"CL0063","aliL":255,"alihindex":"415","is_reported":1,"alintseq":"","jenv":248,"alimmline":"","alihmmacc":"PF13561.9","oasc":"0.95","aliaseq":"SGIGLATVRRFAAEGAKVVVADIDAAA---GEAAASEVGGLFVKVDVTDEAQVEAMFQTAVDTYGGLDIAFNNAGISPPEDDSILTTGIEAWRRVQEVNLTSVYLCSKYAINh

### Put the results in table

In [15]:
# create an empty DataFrame to store the results
results_df = pd.DataFrame()

#for i in range(len(df['meso_seq'])):
for i in range(0,5):
    sequence = df['meso_seq'][i]
    url = 'https://www.ebi.ac.uk/Tools/hmmer/search/hmmscan'
    headers = {'Content-Type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
    data = {'hmmdb': 'pfam',
            'seq': f'>seq\n{sequence}'
                   }

    data = urllib.parse.urlencode(data).encode('ascii')

    response = requests.post(url, headers=headers, data=data, allow_redirects=False)

    redirect_url = response.headers.get('Location')


    if redirect_url is None:
        print("Error: No redirect URL found in response.")
    else:
        response2 = requests.get(redirect_url, headers=headers)
        #print(response2.text)
        results = response2.json()
        hits = results['results']['hits']
        dfff = pd.json_normalize(hits, 'domains', ['acc', 'name', 'score', 'evalue', 'pvalue', 'desc'])
        dfff.insert(0, 'sequence', sequence)
        dfff = dfff.set_index('sequence')
        results_df = pd.concat([results_df, dfff])
        
# print the final results DataFrame
results_df


,alisqacc,aliIdCount,alirfline,is_included,alihmmname,bitscore,display,ievalue,alisqto,aliSim,...,score,evalue,pvalue,desc,act_site.ECH_1.112.E,act_site.ECH_1.132.E,act_site.adh_short_C2.154.Y,act_site.adh_short.158.K,act_site.adh_short.154.Y,act_site.Epimerase.117.Y
sequence,,,,,,,,,,,,,,,,,,,,,
MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTS,,35,,1,DCD,28.693352,0.0,6.0e-07,111,0.660550,...,46.2,0.0,-36.480112,2'-deoxycytidine 5'-triphosphate deaminase (DCD),NaN,NaN,NaN,NaN,NaN,NaN
MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTS,,23,,0,DCD,15.748980,NaN,0.0051,175,0.672131,...,46.2,0.0,-36.480112,2'-deoxycytidine 5'-triphosphate deaminase (DCD),NaN,NaN,NaN,NaN,NaN,NaN
MLLSDRDLVSEIKSGDLSLEPFEPALLQPSSIDVRLDRFFRVFNNHLYTHIDPAEQQDDLTAEVEVTDGEAFVLHPGEFVLASTLEVITLGDQLAGRLEGKSSLGRLGLLTHSTAGFIDPGFSGHVTLELSNVANLPIKLWPGMKIGQLCIFRLSSPAEHPYGSAVYGSRYQGQRGPTPSRSAQNFRLWPTS,,30,,1,dUTPase,40.322865,1.0,0.0,178,0.663636,...,40.7,0.0,-32.277306,dUTPase,NaN,NaN,NaN,NaN,NaN,NaN
MRFEGTSGYVATDDLKVAVNAAIALERPLLVKGEPGTGKTVLAVEVAKAVGAPLIEWHVKSTTKALQGLYEYDAVSRLRDSQLGDERVKDIRNYIKRGKLWEAFVADERPVLLIDEIDKADIEFPNDLLQELDRMEFYVYETGETVRARRRPIVVITSNNEKELPDAFLRRCFFHYIRFPDADTMRAIVDVHFPGIKPRLVSEALKLFYEIRDVPGMKKKPSTSELLDWLKLLMVEDIGPETLRERDPKRLIPPLHGALLKNEQDVHLFERLAFMARREGR,,41,,1,AAA,51.984688,1.0,1.0e-13,175,0.690476,...,52.6,0.0,-40.228785,ATPase family associated with various cellular...,NaN,NaN,NaN,NaN,NaN,NaN
MRFEGTSGYVATDDLKVAVNAAIALERPLLVKGEPGTGKTVLAVEVAKAVGAPLIEWHVKSTTKALQGLYEYDAVSRLRDSQLGDERVKDIRNYIKRGKLWEAFVADERPVLLIDEIDKADIEFPNDLLQELDRMEFYVYETGETVRARRRPIVVITSNNEKELPDAFLRRCFFHYIRFPDADTMRAIVDVHFPGIKPRLVSEALKLFYEIRDVPGMKKKPSTSELLDWLKLLMVEDIGPETLRERDPKRLIPPLHGALLKNEQDVHLFERLAFMARREGR,,41,,1,AAA_5,43.017456,0.0,0.0,171,0.669118,...,43.6,0.0,-34.13238,AAA domain (dynein-related subfamily),NaN,NaN,NaN,NaN,NaN,NaN
MRFEGTSGYVATDDLKVAVNAAIALERPLLVKGEPGTGKTVLAVEVAKAVGAPLIEWHVKSTTKALQGLYEYDAVSRLRDSQLGDERVKDIRNYIKRGKLWEAFVADERPVLLIDEIDKADIEFPNDLLQELDRMEFYVYETGETVRARRRPIVVITSNNEKELPDAFLRRCFFHYIRFPDADTMRAIVDVHFPGIKPRLVSEALKLFYEIRDVPGMKKKPSTSELLDWLKLLMVEDIGPETLRERDPKRLIPPLHGALLKNEQDVHLFERLAFMARREGR,,9,,0,Sigma54_activat,12.988415,NaN,0.069,48,0.750000,...,26.6,0.000005,-22.166128,Sigma-54 interaction domain,NaN,NaN,NaN,NaN,NaN,NaN
MRFEGTSGYVATDDLKVAVNAAIALERPLLVKGEPGTGKTVLAVEVAKAVGAPLIEWHVKSTTKALQGLYEYDAVSRLRDSQLGDERVKDIRNYIKRGKLWEAFVADERPVLLIDEIDKADIEFPNDLLQELDRMEFYVYETGETVRARRRPIVVITSNNEKELPDAFLRRCFFHYIRFPDADTMRAIVDVHFPGIKPRLVSEALKLFYEIRDVPGMKKKPSTSELLDWLKLLMVEDIGPETLRERDPKRLIPPLHGALLKNEQDVHLFERLAFMARREGR,,13,,0,Sigma54_activat,11.348754,NaN,0.22,177,0.685714,...,26.6,0.000005,-22.166128,Sigma-54 interaction domain,NaN,NaN,NaN,NaN,NaN,NaN
MAYETINVDVQDHVCLIKLHRPEALNALNAALVSELCTALEEADASDKVRCIVLTGSEKAFAAGADIKEMSEMSFTDVYSSNLFAQVNDRIIAIRKPIIAAVAGYALGGGCELAMLCDFIIAAETAKFGQPEINLGVVAGIGGTQRLTRFVGKSKSMDMNLTGRFMGAEEAERAGLVSRVVPAKKLVDEAMAAAHKIAEKSQLTAVAVKEAVNRSYELTLSEGMLFERRAFHSMFATEDQKEGMAAFLEKREAQFRDK,,117,,1,ECH_1,275.810150,1.0,0.0,258,0.870968,...,276.0,0.0,-197.688691,Enoyl-CoA hydratase/isomerase,Similarity to Q5LLW6,Similarity to Q5LLW6,NaN,NaN,NaN,NaN
MAYETINVDVQDHVCLIKLHRPEALNALNAALVSELCTALEEADASDKVRCIVLTGSEKAFAAGADIKEMSEMSFTDVYSSNLFAQVNDRIIAIRKPIIAAVAGYALGGGCELAMLCDFIIAAETAKFGQPEINLGVVAGIGGTQRLTRFVGKSKSMDMNLTGRFMGAEEAERAGLVSRVVPAKKLVDEAMAAAHKIAEKSQLTAVAVKEAVNRSYELTLSEGMLFERRAFHSMFATEDQKEGMAAFLEKREAQFRDK,,60,,1,ECH_2,119.056747,0.0,0.0,186,0.755814,...,129.0,3.0e-37,-93.973921,Enoyl-CoA hydratase/isomerase,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
results_df.iloc[5]

alisqacc                                                       
aliIdCount                                                    9
alirfline                                                      
is_included                                                   0
alihmmname                                      Sigma54_activat
bitscore                                              12.988415
display                                                     NaN
ievalue                                                   0.069
alisqto                                                      48
aliSim                                                     0.75
jali                                                         48
bias                                                       0.02
ienv                                                          9
cevalue                                                0.000011
significant                                                 NaN
alimline                         ++++++ 